In [95]:
%load_ext autoreload
%autoreload 2

import defs as d
import argparse
import os
import sys
import gc
from azureml.core import Run
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import json
import joblib
import lightgbm as lgb
sys.path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['',
 '/home/jovyan/dev/projects/vinx',
 '/opt/conda/envs/azureml/lib/python36.zip',
 '/opt/conda/envs/azureml/lib/python3.6',
 '/root/.ipython',
 '/opt/conda/envs/azureml/lib/python3.6/lib-dynload',
 '/opt/conda/envs/azureml/lib/python3.6/site-packages',
 '/opt/conda/envs/azureml/lib/python3.6/site-packages/IPython/extensions',
 '/home/jovyan/dev/projects/vinx/work/lib',
 '/home/jovyan/dev/projects/vinx/work/datasource',
 '/home/jovyan/dev/projects/vinx/work/datasource/interim',
 '/home/jovyan/dev/projects/vinx/work/datasource/original',
 '/home/jovyan/dev/projects/vinx/work/datasource/other',
 '/home/jovyan/dev/projects/vinx/work/lib/vinx_azure/vinx_azure_ml',
 '/home/jovyan/dev/projects/vinx/work/src',
 '/home/jovyan/dev/projects/vinx/work/src/azure_src',
 '/home/jovyan/dev/projects/vinx/work/lib/vinx_azure/vinx_azure_ml',
 '/opt/conda/envs/azureml/lib/python3.6/site-packages/azureml/_project/vendor',
 '/opt/conda/envs/azureml/lib/python3.6/site-packages/azureml/automl/core/_vendor'

In [96]:
%%time
print('start')
import cross_validation as cv
cv

start
CPU times: user 0 ns, sys: 1.02 ms, total: 1.02 ms
Wall time: 747 µs


<module 'cross_validation' from '/home/jovyan/dev/projects/vinx/work/lib/vinx_azure/vinx_azure_ml/cross_validation.py'>

In [97]:
%%time
print('start')

data = pd.read_pickle(d.TMP+'/merged_data.pkl')
data

start
CPU times: user 56.4 ms, sys: 305 ms, total: 362 ms
Wall time: 637 ms


,store_cd,date,customer_count,maxtemp,mintemp,rain,post_cd,year,month,day,...,lag_28,lag_29,lag_30,expanding_mean,ma7,ma14,ma30,std7,std14,std30
0,113,2017-01-27,3087,12.7,3.1,0.0,6730041,2017,1,27,...,NaN,NaN,NaN,3087.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,113,2017-01-28,2309,10.8,4.5,0.0,6730041,2017,1,28,...,NaN,NaN,NaN,2698.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,113,2017-01-29,2174,10.2,3.8,3.5,6730041,2017,1,29,...,NaN,NaN,NaN,2523.333333,NaN,NaN,NaN,NaN,NaN,NaN
3,113,2017-01-30,3248,14.3,4.5,1.0,6730041,2017,1,30,...,NaN,NaN,NaN,2704.500000,NaN,NaN,NaN,NaN,NaN,NaN
4,113,2017-01-31,3003,7.1,2.2,0.0,6730041,2017,1,31,...,NaN,NaN,NaN,2764.200000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210953,881,2020-05-07,2398,24.1,13.5,0.0,5798051,2020,5,7,...,2486.0,2426.0,2535.0,1062.777778,2208.285714,2233.285714,2232.066667,159.716117,171.261127,273.222144
210954,881,2020-05-08,2476,23.2,12.5,0.0,5798051,2020,5,8,...,2477.0,2486.0,2426.0,1078.307692,2219.857143,2243.571429,2233.733333,177.448047,181.654666,274.594991
210955,881,2020-05-09,2219,22.9,15.6,1.5,5798051,2020,5,9,...,2693.0,2477.0,2486.0,1090.706522,2217.857143,2240.142857,2224.833333,177.354098,181.631435,270.432104
210956,881,2020-05-10,1984,26.2,17.3,4.0,5798051,2020,5,10,...,1319.0,2693.0,2477.0,1100.311828,2212.428571,2242.428571,2208.400000,184.774509,177.889211,269.557952


In [98]:
%%time
print('start')
opt_horizon = 30
pred_horizon = 90
n_jobs = 1

start
CPU times: user 706 µs, sys: 1.07 ms, total: 1.77 ms
Wall time: 1.62 ms


In [99]:
%%time
print('start')

# 27日*12ヶ月 = 324
split_masks = cv._split_masks(data, 324, opt_horizon, 1)

start
CPU times: user 2.11 s, sys: 1.58 ms, total: 2.11 s
Wall time: 2.11 s


In [100]:
%%time
print('start')

best_parameter = {
  'objective': 'gamma',
  'metric': 'fair',
  'verbosity': -1,
  'boosting_type': 'gbdt',
  # Bagging will randomly select part of data without resampling.
  # With time-series nature of our problem, bagging should be disabled.
  'bagging_fraction': 1,
  'bagging_freq': 0,

  # This parameter should not be zero (meaning no limit).
  # Need to find the best interval.
  'max_depth': 0,

  # Shrinkage rate
  'learning_rate': 0.0906830423216638,

  # Take boosting_round large and early-stop if the metric doesn't improve.
  # 'num_boost_round': trial.suggest_int('num_boost_round', 100, 200),
  'num_boost_round': 500,
  'early_stopping': 5,

  # Each value of `store_cd` should have its own branch.
  # If you accept this claim, there should at least be 185 leaves(region).
  # If you split 2 region for each feature, then there will be 2^num_features regions(leaves).
  # Sample once in a 10 interval 901-910, 911-920, ...
  'num_leaves': 468,

  # Regularization parameters.
  # p(0.1 ~ 0.01) = 1/5, p(0.01 ~ 0.001) = 1/5, ... , p(0.00001 ~ 0.000001) = 1/5
  # 10 sampling in each of above interval
  'lambda_l1': 0.05026988990259302,
  'lambda_l2': 0.09999453004404879,

  # Fraction of features that lgbm uses for each training.
  # Lower value means faster training.
  # seed value is the default = 2
  'feature_fraction': 0.7830944244992202,
  'feature_fraction_seed': 2,

  # This parameter is for ensuring LLN(Law of Large Numbers) and should not randomly be tuned.
  #
  'min_data_in_leaf': 28,
}
estimator = lgb

start
CPU times: user 275 µs, sys: 10 µs, total: 285 µs
Wall time: 263 µs


In [106]:
%%time
print('start')

result = []
for item in split_masks:
  train = data[item[0]]
  valid = data[item[1]]
  split_date = valid['date'].min()
  print(split_date)
  pred_train, pred_valid = cv._split(data, split_date, pred_horizon)
  pred_data = data[pred_valid]

  train_x = train.drop(['customer_count', 'date'], axis=1).copy().to_numpy()
  train_y = train['customer_count'].copy().to_numpy()
  valid_x = valid.drop(['customer_count', 'date'], axis=1).copy().to_numpy()
  valid_y = valid['customer_count'].copy().to_numpy()

  if valid_y.shape[0] != np.count_nonzero(valid_y):
    pass

  # dtrain = lgb.Dataset(train_x, label=train_y)
  # dvalid = lgb.Dataset(valid_x, valid_y, reference=dtrain)
  # model = estimator.train(best_parameter, dtrain, valid_sets=[dtrain, dvalid], verbose_eval=-1)
  # preds = np.rint(model.predict(valid_x, num_iteration=model.best_iteration))
  #
  # use_cols = ['store_cd', 'date', 'customer_count']
  # pred_df = valid[use_cols].copy()
  # pred_df['preds'] = preds
  # pred_df['split_date'] = split_date
  # result.append(pred_df)


start
2019-01-28 00:00:00
     store_cd       date  customer_count  maxtemp  mintemp  rain  post_cd  \
731       113 2019-01-28            2882     10.5      2.8   0.5  6730041   
732       113 2019-01-29            2758      8.1      2.7   0.0  6730041   
733       113 2019-01-30            3336      9.8      2.4   0.0  6730041   
734       113 2019-01-31            2632      8.0      4.1  12.0  6730041   
735       113 2019-02-01            3574      7.7      2.1   0.0  6730041   
..        ...        ...             ...      ...      ...   ...      ...   
817       113 2019-04-24            2943     22.1     18.3   2.0  6730041   
818       113 2019-04-25            3313     23.3     17.6   0.0  6730041   
819       113 2019-04-26            3045     23.0     13.6   4.5  6730041   
820       113 2019-04-27            2233     15.9      9.6   0.5  6730041   
821       113 2019-04-28            2018     16.0      8.0   0.0  6730041   

     year  month  day  ...  lag_28  lag_29  lag_3

/opt/conda/envs/azureml/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/envs/azureml/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/envs/azureml/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/envs/azureml/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/envs/azureml/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index

In [73]:
%%time
print('start')
preds.shape

start
CPU times: user 0 ns, sys: 567 µs, total: 567 µs
Wall time: 582 µs


(5328,)

In [79]:
%%time
print('start')
# Create dataframe with the following specification
# prediction_df.columns = [split_date, prediction_date, store_cd, y, yhat]

use_cols = ['store_cd', 'date', 'customer_count']
pred_df = valid[use_cols].copy()
pred_df['preds'] = preds
pred_df['split_date'] = split_date
use_cols = ['split_date', 'store_cd', 'date', 'customer_count', 'preds']
pred_df[use_cols]

start
CPU times: user 6.34 ms, sys: 0 ns, total: 6.34 ms
Wall time: 5.38 ms


,split_date,store_cd,date,customer_count,preds
731,2019-01-28,113,2019-01-28,2882,2660.0
732,2019-01-28,113,2019-01-29,2758,2894.0
733,2019-01-28,113,2019-01-30,3336,2967.0
734,2019-01-28,113,2019-01-31,2632,2869.0
735,2019-01-28,113,2019-02-01,3574,3190.0
...,...,...,...,...,...
202804,2019-01-28,861,2019-02-23,1751,1721.0
202805,2019-01-28,861,2019-02-24,1814,1799.0
202806,2019-01-28,861,2019-02-25,1798,1812.0
202807,2019-01-28,861,2019-02-26,1814,1812.0


In [94]:
%%time
print('start')

delta =
delta.astype(int)


start
CPU times: user 2.34 ms, sys: 1.08 ms, total: 3.43 ms
Wall time: 3 ms


731        0
732        1
733        2
734        3
735        4
          ..
202804    26
202805    27
202806    28
202807    29
202808    30
Name: date, Length: 5328, dtype: int64

In [87]:
%%time
print('start')
pred_df['date']

start
CPU times: user 133 µs, sys: 1.05 ms, total: 1.18 ms
Wall time: 721 µs


731      2019-01-28
732      2019-01-29
733      2019-01-30
734      2019-01-31
735      2019-02-01
            ...    
202804   2019-02-23
202805   2019-02-24
202806   2019-02-25
202807   2019-02-26
202808   2019-02-27
Name: date, Length: 5328, dtype: datetime64[ns]

In [66]:
%%time
print('start')
importance = pd.DataFrame(model.feature_importance(), index=train.drop(['customer_count', 'date'], axis=1).columns, columns=['importance'])
importance.sort_values('importance', ascending=False)

start
CPU times: user 13.1 ms, sys: 10.9 ms, total: 24 ms
Wall time: 21.2 ms


,importance
ma7,13791
std7,10922
lag_1,10214
lag_2,8239
lag_3,8179
lag_5,7986
lag_4,7981
lag_6,7919
day,7096
maxtemp,5986
